<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/spike_latency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####################### SPIKE LATENCY #######################

def Spike_latency(abf_recordings_df, protocol_aliases,to_plot=False):
    'Loops through abfs and calcualtes the time to first action potential'
    'during a ramp current stimulation'


    correct_protocol = [ p in protocol_aliases for p in abf_recordings_df['protocol']]
    # print(np.sum(correct_protocol),'files to analyze...')
    for file_name in tqdm( abf_recordings_df.index[correct_protocol]): #tqdm( ) : 
        abf = pyabf.ABF( file_name )
        # pass_rate, passing_sweeps, QC_check_df, QC_val_df = Vclamp_QC(file_name,to_plot=False,verbose=False)
        passing_sweeps=abf_recordings_df.at[file_name,'passing_sweeps']
        latencey_list = []
        v_hold_list = []
        

        for s in abf.sweepList:
            abf.setSweep(s)
            # plot_sweeps_and_command(abf)
            latencey, v_hold = analyze_ramp_sweep(abf.sweepX,abf.sweepY,abf.sweepC,
                                              abf.sampleRate,to_plot=to_plot)
            latencey_list.append(latencey)
            v_hold_list.append(v_hold)
        latencey_list =np.array(latencey_list)
        v_hold_list =np.array(v_hold_list)
        # if len(passing_sweeps)==0:
        #     abf_recordings_df.at[file_name,'Spike_Latency_(ms)'] = np.nan
        #     abf_recordings_df.at[file_name,'V_hold_(Latency)'] = np.nan
        # else:      
        # #     latencey_list = latencey_list[passing_sweeps]
        # #     v_hold_list = v_hold_list[passing_sweeps]
        abf_recordings_df.at[file_name,'Spike_Latency_(ms)'] = np.median(latencey_list)
        abf_recordings_df.at[file_name,'V_hold_(Latency)'] = np.median(v_hold_list)
    return abf_recordings_df
        
        
def analyze_ramp_sweep(sweepX,sweepY,sweepC,rate,to_plot=False):
    'Receives sweep data and finds the first AP and returns it.'
    'Also retuns Vhold for quality control.'
    is_base = sweepC==sweepC[0]
    is_stim = np.logical_not(sweepC==sweepC[0])
    ramp_start_ind = np.min(np.where(is_base==False))
    v_hold = np.mean( sweepY[0:ramp_start_ind])
    # print(sweepX,sweepY)

    spike_args = {'spike_thresh':10,
                    'high_dv_thresh': 25,
                    'low_dv_thresh': -5,
                     'window_ms': 2}

    dVds, over_thresh, inds, mean_spike_rate = find_spike_in_trace(sweepY, rate,spike_args,is_stim=is_stim)
 
    if len(inds)==0:
        # print('no spikes found')
        return np.nan,v_hold
    latencey = sweepX[np.min(inds)-ramp_start_ind]*1000
    if to_plot:
        # plt.scatter(sweepX,dVds,color='k')
        plt.plot(sweepX,sweepY,color='k')
        plt.scatter(sweepX[inds],sweepY[inds],color='r' )
        zoom_x_relativ = np.array([ 0.75, 1.5])
        zoom_x = zoom_x_relativ*(latencey/1000+sweepX[ramp_start_ind])
        # print('zoom_x',zoom_x)
        # print('latencey',latencey)
        # print('sweepX[ramp_start_ind]',sweepX[ramp_start_ind])
        # print('ramp_start_ind',ramp_start_ind)
        plt.gca().set_xlim(zoom_x)
        plt.show()

    return latencey, v_hold